In [ ]:
# ── Google Colab Setup ────────────────────────────────────────────────────────
# This cell is a no-op when running locally.
import sys, os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from pathlib import Path
    import subprocess

    repo_dir = Path('/content/cryptoTrading2')

    # 1. Clone repo (skipped if already present)
    if not repo_dir.exists():
        subprocess.run(
            ['git', 'clone', 'https://github.com/kingsuching/cryptoTrading2.git', str(repo_dir)],
            check=True,
        )

    # 2. Set working directory and Python path
    os.chdir(str(repo_dir))
    if str(repo_dir) not in sys.path:
        sys.path.insert(0, str(repo_dir))

    # 3. Install dependencies
    subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-q',
         'pandas', 'numpy', 'scikit-learn', 'statsmodels', 'prophet',
         'torch', 'matplotlib', 'tqdm', 'transformers',
         'beautifulsoup4', 'requests', 'serpapi'],
        check=True,
    )

    # 4. Write API keys from Colab Secrets (add CMC_KEY & SERPAPI_KEY in the key icon sidebar)
    try:
        from google.colab import userdata
        cmc     = userdata.get('CMC_KEY')     or ''
        serpapi = userdata.get('SERPAPI_KEY') or ''
    except Exception:
        cmc, serpapi = '', ''
    with open('files/API_KEYS.py', 'w') as _f:
        _f.write(f'CMC_KEY = "{cmc}"\nSERPAPI_KEY = "{serpapi}"\n')
    if not cmc or not serpapi:
        print("NOTE: API keys not set — add CMC_KEY & SERPAPI_KEY in Colab Secrets if data fetching is needed.")

    print(f"Colab setup complete. CWD: {os.getcwd()}")

In [1]:
import os, sys
from pathlib import Path

_root = next((p for p in [Path(os.getcwd()), *Path(os.getcwd()).parents]
              if p.name == 'cryptoTrading2'), None)
if _root:
    os.chdir(str(_root))
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

import warnings
warnings.filterwarnings('ignore')

from files.functions import predict_matrix
from files.CONSTANTS import COIN

# Predictions Matrix

Loads every saved model artifact for `COIN` and produces an **m × n** prediction matrix  
where **m** = number of models (8) and **n** = `TEST_DAYS` (7).

Models: GBM · SVM · KNN · ARIMA · LSTM · TFT · Transformer · Prophet

In [ ]:
matrix = predict_matrix(COIN)

print(f"Prediction matrix — {COIN}  ({matrix.shape[0]} models × {matrix.shape[1]} days)\n")
print(matrix.round(2).to_string())

In [ ]:
# ── Heatmap ───────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt

vals = matrix.values.astype(float)

fig, ax = plt.subplots(figsize=(12, 5))
im = ax.imshow(vals, aspect='auto', cmap='RdYlGn')

ax.set_xticks(range(len(matrix.columns)))
ax.set_xticklabels(matrix.columns, rotation=30, ha='right', fontsize=9)
ax.set_yticks(range(len(matrix.index)))
ax.set_yticklabels(matrix.index, fontsize=10)

for i in range(vals.shape[0]):
    for j in range(vals.shape[1]):
        ax.text(j, i, f'{vals[i, j]:,.0f}', ha='center', va='center',
                fontsize=8, color='black')

plt.colorbar(im, ax=ax, label='Predicted Price (USD)')
ax.set_title(f"{COIN} — 7-Day Price Forecast by Model", fontsize=13, pad=12)
plt.tight_layout()
plt.show()